# CNN on MNIST

This tutorial demonstrates how to build a simple convolutional neural network (CNN) with Inox, and train it to classify digits. It is intended for those who are new to JAX and Inox, or simply curious.

Unlike [PyTorch](https://pytorch.org), which is a centralized framework, the [JAX](https://jax.readthedocs.io) ecosystem is distributed as a collection of packages, each tackling a well-defined task. This tutorial uses [Inox](https://inox.readthedocs.io) to build the network, [Optax](https://optax.readthedocs.io) to optimize the network, and [PyTorch](https://pytorch.org/) to load the [MNIST](https://wikipedia.org/wiki/MNIST_database) data.

In [1]:
# !pip install jax[cpu] inox optax tqdm
# !pip install torchvision --extra-index-url https://download.pytorch.org/whl/cpu

import jax
import inox
import inox.nn as nn
import optax

from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_tensor
from tqdm import tqdm

jax.config.update('jax_platform_name', 'cpu')
jax.numpy.set_printoptions(suppress=True)

## Data

JAX does not provide built-in datasets and dataloaders, as there are already many alternatives. We load the MNIST dataset using PyTorch.

In [2]:
trainset = MNIST('data', train=True, download=True, transform=to_tensor)
testset = MNIST('data', train=False, download=True)

loader = DataLoader(trainset, batch_size=8, shuffle=True, drop_last=True)

In [3]:
x, y = next(iter(loader))
x, y = x.numpy(), y.numpy()

print(x.shape)
print(y)

(8, 1, 28, 28)
[5 4 6 3 9 5 3 7]


In the above example, `x` is a batch of black and white images and `y` is a batch of corresponding digits.

## Network

Our network is a simple convolutional neural network. We define its architecture by a sequence of operations, often called layers. These operations do not need to be Inox modules, they can be any callables.

A few remarks:

1. `nn.Rearrange` is a small wrapper around [einops](https://github.com/arogozhnikov/einops)'s `rearrange`, and enables intuitive and efficient axis manipulations.
2. Like TensorFlow, Inox adopts a channel-last convention for axes, meaning that a batch of images is expected to have a shape $(N, H, W, C)$, where $C$ is the number of channels.
3. Some layers, like `nn.Linear` and `nn.Conv`, require a random number generator (RNG) key for initialization.

In [4]:
class CNN(nn.Module):
    def __init__(self, key):
        keys = jax.random.split(key, 4)

        self.layers = [
            nn.Rearrange('C H W -> H W C'),
            nn.Conv(keys[0], spatial=2, in_channels=1, out_channels=4),
            nn.ReLU(),
            nn.Conv(keys[1], spatial=2, in_channels=4, out_channels=4),
            nn.MaxPool(spatial=2, window_size=2),
            nn.Rearrange('H W C -> (H W C)'),
            nn.Linear(keys[2], in_features=576, out_features=256),
            nn.ReLU(),
            nn.Linear(keys[3], in_features=256, out_features=10),
        ]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)

        return x

    @jax.jit
    def predict(self, x):
        return jax.nn.softmax(self(x))

Our network is a PyTree, that is just a nested collection of Python objects. Some of these objects are JAX arrays, like the kernel of the first convolution `network.layers[1].kernel`, while others are arbitrary objects, like the string `network.layers[0].pattern`. Inox provides a nice representation for its modules.

In [5]:
network = CNN(jax.random.PRNGKey(0))
network

CNN(
  layers = [
    Rearrange(
      lengths = {},
      pattern = 'C H W -> H W C'
    ),
    Conv(
      bias = float32[4],
      dilation = [1, 1],
      groups = 1,
      kernel = float32[3, 3, 1, 4],
      kernel_size = [3, 3],
      padding = [(0, 0), (0, 0)],
      spatial = 2,
      stride = [1, 1]
    ),
    ReLU(),
    ...,
    Linear(
      bias = float32[256],
      weight = float32[576, 256]
    ),
    ReLU(),
    Linear(
      bias = float32[10],
      weight = float32[256, 10]
    )
  ]
)

Now that our network is built, we can use it to make predictions. However, since it has not been trained yet, it is currently unable to classify the digits. In the next cell, you see that the probability it associates with each digit (0 to 9) is more or less uniform.

In [6]:
network.predict(x[0])

Array([0.10095079, 0.09911551, 0.10107352, 0.09597864, 0.10047358,
       0.10346892, 0.09940311, 0.09472179, 0.10336623, 0.10144799],      dtype=float32)

We can quantify the quality of our network's predictions with their [cross entropy](https://wikipedia.org/wiki/Cross_entropy). For perfect predictions, the cross entropy is null, making it a good training objective.

In [7]:
def cross_entropy(f, x, y):
    return optax.softmax_cross_entropy_with_integer_labels(
        logits=f(x),
        labels=y,
    )

In [8]:
cross_entropy(jax.vmap(network), x, y)

Array([2.2684839, 2.300259 , 2.234749 , 2.348772 , 2.3462443, 2.245705 ,
       2.3467388, 2.3620248], dtype=float32)

## Training

Now that we have an objective to minimize, we can start to train the parameters $\phi$ of our network. Inox modules have a special `partition` method which separates arrays that can be optimized, typically parameters, from those that should not. We initialize an Optax optimizer (Adam) for the parameters of our network.

In [9]:
params, _, _ = network.partition()
optimizer = optax.adam(learning_rate=1e-3)
opt_state = optimizer.init(params)

A training step consists in computing the gradients of the loss $\ell(\phi)$, here the cross entropy, with respect to the parameters $\phi$ and updating the parameters according to the gradients. The whole procedure is written as a single JIT-compiled function to make it run as fast as possible. 

In [10]:
@jax.jit
def step(network, opt_state, x, y):
    params, buffers, build = network.partition()

    def ell(params):
        return cross_entropy(jax.vmap(build(params, buffers)), x, y).mean()

    loss, grads = jax.value_and_grad(ell)(params)
    updates, opt_state = optimizer.update(grads, opt_state, params)
    params = optax.apply_updates(params, updates)

    return build(params, buffers), opt_state

Then, to train the network, we iteratively apply our training step with random batches loaded from our training set.

In [11]:
for x, y in tqdm(loader):
    x, y = x.numpy(), y.numpy()
    network, opt_state = step(network, opt_state, x, y)

100%|██████████| 7500/7500 [00:35<00:00, 211.81it/s]


## Evaluation

Now that the network is trained, we can use it to make predictions.

In [12]:
x, _ = testset[0]
x

In [13]:
network.predict(to_tensor(x).numpy())

Array([0.00000007, 0.00000002, 0.00000499, 0.00000303, 0.        ,
       0.0000001 , 0.        , 0.9999918 , 0.        , 0.00000015],      dtype=float32)